In [1]:
from neural_lam.datastore import init_datastore

datastore_kind = 'mdp'
config_path = 'C:/Users/23603526/Documents/GitHub/neural-lam-Divan/era5/era.datastore.yaml'

datastore = init_datastore(
        datastore_kind=datastore_kind,
        config_path = config_path,
    )

2025-01-20 10:38:12.459 | INFO     | mllam_data_prep.create_dataset:create_dataset:135 - Loading dataset era_variables from C:\Users\23603526\Documents\GitHub\neural-lam-Divan\era5\era5_2010_2019_xy.nc
c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)
2025-01-20 10:38:14.136 | INFO     | mllam_data_prep.create_dataset:create_dataset:160 - Mapping dimensions and variables for dataset era_variables to state
2025-01-20 10:38:40.659 | INFO     | mllam_data_prep.create_dataset:create_dataset:135 - Loading dataset era_times from C:\Users\23603526\Documents\GitHub\neural-lam-Divan\era5\era5_2010_2019_xy.nc
2025-01-20 10:38:40.709 | INFO     | mllam_data_prep.create_dataset:create_dataset:160 - Mapping dimensions and variables for dataset era_times to forcing
2025-01-20 10:38:53.316 | I

The loaded datastore contains the following features:
 state   : z850.0hPa r850.0hPa t850.0hPa u850.0hPa v850.0hPa w850.0hPa wind_speed850.0hPa
 forcing : hour month
 static  : lsm
With the following splits (over time):
 train   : 2010-01-01T00:00 to 2017-06-01T00:00
 val     : 2017-07-01T00:00 to 2018-11-01T00:00
 test    : 2018-12-01T00:00 to 2019-12-31T00:00


In [2]:
# Open the datastore era5\era.datastore.zarr

import xarray as xr

ds = xr.open_dataset('/teamspace/studios/this_studio/dk-neural-lam/era5/era.datastore.zarr', engine='zarr')

ds

<xarray.Dataset> Size: 2GB
Dimensions:                         (forcing_feature: 2, time: 14605,
                                     grid_index: 3381, split_name: 3,
                                     split_part: 2, state_feature: 7,
                                     static_feature: 1)
Coordinates: (12/17)
  * forcing_feature                 (forcing_feature) <U5 40B 'hour' 'month'
    forcing_feature_long_name       (forcing_feature) <U4 32B ...
    forcing_feature_source_dataset  (forcing_feature) <U9 72B ...
    forcing_feature_units           (forcing_feature) <U1 8B ...
  * split_name                      (split_name) <U5 60B 'train' 'val' 'test'
  * split_part                      (split_part) <U5 40B 'start' 'end'
    ...                              ...
    static_feature_long_name        (static_feature) <U1 4B ...
    static_feature_source_dataset   (static_feature) <U7 28B ...
    static_feature_units            (static_feature) <U1 4B ...
  * time                            (time) datetime64[ns] 117kB 2010-01-01 .....
    x                               (grid_index) float64 27kB ...
    y                               (grid_index) float64 27kB ...
Dimensions without coordinates: grid_index
Data variables: (12/14)
    forcing                         (forcing_feature, time, grid_index) int64 790MB ...
    forcing__train__diff_mean       (forcing_feature) float64 16B ...
    forcing__train__diff_std        (forcing_feature) float64 16B ...
    forcing__train__mean            (forcing_feature) float64 16B ...
    forcing__train__std             (forcing_feature) float64 16B ...
    splits                          (split_name, split_part) <U16 384B ...
    ...                              ...
    state__train__diff_std          (state_feature) float32 28B ...
    state__train__mean              (state_feature) float32 28B ...
    state__train__std               (state_feature) float32 28B ...
    static                          (static_feature, grid_index) float32 14kB ...
    static__train__mean             (static_feature) float32 4B ...
    static__train__std              (static_feature) float32 4B ...
Attributes:
    created_on:       2025-01-20T10:39:12
    created_with:     mllam-data-prep (https://github.com/mllam/mllam-data-prep)
    dataset_version:  v0.1.0
    mdp_version:      v0.5.0
    schema_version:   v0.5.0